In [0]:
import pandas as pd

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Leer Parquet").getOrCreate()
base_path = 'abfss://bronze@storedatalakedwh.dfs.core.windows.net/teradata/cobertura.parquet'
df_spark = spark.read.parquet(base_path)

In [0]:
df_est1 = df_spark.filter(df_spark.ESTRATO_CATASTRAL=='1').orderBy("Fecha")

In [0]:
df = df_est1.toPandas()

In [0]:
df[df['Fecha']=='2021-06-01']

In [0]:
# Agrupar por fecha y contar los registros y los '1' en la columna 'cta'
df_agrupado = df.groupby('Fecha').agg(
    total_registros=('Cta', 'count'),             # Contar todos los registros
    total_cta=('Cta', lambda x: (x == 1).sum())  # Contar los 1 en 'cta'
).reset_index()

df_agrupado['%llenado'] = df_agrupado['total_cta']*100 / df_agrupado['total_registros']

df_agrupado

In [0]:
# Unir df_agrupado a df
df_unido = pd.merge(df, df_agrupado, on='Fecha', how='left')
df_unido